# Final ML models over ten random seeds - after count binarization

## Outline

The **MLAging - all-cell** workflow consists of sections:

`00 preprocessing.R` Data preprocessing and preparation in Seurat.

`111 All-cell Model Tuning - Before Binarization` ML model tunning using *non-binarized* HVGs and hyperparameter selection using `GridSearchCV`.

`112 All-cell Model Tuning - After Binarization` ML model tunning using *binarized* HVGs.

`121 All-cell Model 10x - Before Binarization` Run the best models for non-binarized* HVGs over 10 random seeds.

`122 All-cell Model 10x - After Binarization` Run the best models for *binarized* HVGs over 10 random seeds -- **this notebook**:

- [1. Lasso - L1](#1.-l1)
- [2. Ridge - L2](#2.-l2)
- [3. ElasticNet](#3.-eln)
    
    
- [4. Random Forest](#4.-rfc)
- [5. XGBoost](#5.-xgbc)
    
    
- [6. Support Vector Machine with rbf kernel](#6.-svc)
 
`123 All-cell Model 10x Swapped Train-Test` Run the best models for *binarized* HVGs over 10 random seeds. But switched the training and test sets to make sure that the sequencing throughput did not affect model performance.

`13 All-cell Model Result Viz` Result visulization.

`14 All-cell ELN Interpretation` Result interpretation. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

from src.preprocessing_eln import *
from src.data_processing import *
from src.grid_search import *
import os
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression
import tqdm
from tqdm import tqdm
from statistics import mean, stdev

data_type = 'float32'

In [2]:
pr_auc_scorer = make_scorer(pr_auc_score, greater_is_better=True,
                            needs_proba=True)

In [3]:
input_test = '../data/test_final_group_info.csv'
input_train = '../data/train_final_group_info.csv'

cell_type = 'All'

train_X, train_y, test_X, test_y, custom_cv = data_prep(input_test, input_train,
                                                        cell_type, binarization=True)

Finished data prepration for All


### 1. L1 <a name="1.-l1"></a>

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
scores = []
final_test = []
final_models = []
for i in tqdm(range(10)):
    random_state = 42*i    
    X_test, y_test = shuffle(test_X, test_y, random_state=random_state)
    X_train, y_train = shuffle(train_X, train_y, random_state=random_state)
    
    l1 = LogisticRegression(penalty='l1', C=19, solver='saga', max_iter=10000000)
        
    l1.fit(X_train, y_train)
    
    y_pred = l1.predict_proba(X_test)[:, 1]
    auprc = pr_auc_score(y_test, y_pred)
    
    final_test.append((X_test, y_test))
    final_models.append(l1)
    scores.append(auprc)   
print(f'auprc: {mean(scores)} ± {stdev(scores)}' )

100%|██████████| 10/10 [2:03:49<00:00, 742.91s/it] 

auprc: 0.9607897215308782 ± 2.1466681653137513e-06


In [6]:
file = open('../results/results_bin_best/l1_model_test_scores.save', 'wb')
pickle.dump(scores, file)
file.close()

file = open('../results/results_bin_best/l1_model_test_sets.save', 'wb')
pickle.dump(final_test, file)
file.close()

file = open('../results/results_bin_best/l1_model_test_models.save', 'wb')
pickle.dump(final_models, file)
file.close()

### 2. L2 <a name="2.-l2"></a>

In [7]:
scores = []
final_models = []
for i in tqdm(range(10)):
    random_state = 42*i    
    X_test, y_test = final_test[i]
    X_train, y_train = shuffle(train_X, train_y, random_state=random_state)
    
    l2 = LogisticRegression(penalty='l2', C=0.0774263682681127, solver='saga', max_iter=10000000)
        
    l2.fit(X_train, y_train)
    
    y_pred = l2.predict_proba(X_test)[:, 1]
    auprc = pr_auc_score(y_test, y_pred)
    
    final_models.append(l2)
    scores.append(auprc)   
print(f'auprc: {mean(scores)} ± {stdev(scores)}' )

100%|██████████| 10/10 [13:25<00:00, 80.51s/it]

auprc: 0.966928053303436 ± 1.1663644175156882e-06


In [8]:
file = open('../results/results_bin_best/l2_model_test_scores.save', 'wb')
pickle.dump(scores, file)
file.close()

file = open('../results/results_bin_best/l2_model_test_models.save', 'wb')
pickle.dump(final_models, file)
file.close()

### 3. ELN <a name="3.-eln"></a>

In [9]:
scores = []
final_models = []
for i in tqdm(range(10)):
    random_state = 42*i    
    X_test, y_test = final_test[i]
    X_train, y_train = shuffle(train_X, train_y, random_state=random_state)
    
    eln = LogisticRegression(penalty='elasticnet', C=0.046415888336127774, l1_ratio=0.01, 
                             solver='saga', max_iter=10000000)
        
    eln.fit(X_train, y_train)
    
    y_pred = eln.predict_proba(X_test)[:, 1]
    auprc = pr_auc_score(y_test, y_pred)
    
    final_models.append(eln)
    scores.append(auprc)   
print(f'auprc: {mean(scores)} ± {stdev(scores)}' )

100%|██████████| 10/10 [19:52<00:00, 119.27s/it]

auprc: 0.967162167888304 ± 2.1696209650836154e-06


In [10]:
file = open('../results/results_bin_best/eln_model_test_scores.save', 'wb')
pickle.dump(scores, file)
file.close()

file = open('../results/results_bin_best/eln_model_test_models.save', 'wb')
pickle.dump(final_models, file)
file.close()

### 4. RFC <a name="4.-rfc"></a>

In [12]:
from sklearn.ensemble import RandomForestClassifier

final_test = []
final_models = []
for i in tqdm(range(10)):
    random_state = 42*i    
    X_test, y_test = shuffle(test_X, test_y, random_state=random_state)
    X_train, y_train = shuffle(train_X, train_y, random_state=random_state)
    
    rfc = RandomForestClassifier(max_features=10, max_depth=20, min_samples_split=10)
        
    rfc.fit(X_train, y_train)
    
    y_pred = rfc.predict_proba(X_test)[:, 1]
    auprc = pr_auc_score(y_test, y_pred)
    
    final_test.append((X_test, y_test))
    final_models.append(rfc)
    scores.append(auprc)   
print(f'auprc: {mean(scores)} ± {stdev(scores)}')

100%|██████████| 10/10 [00:30<00:00,  3.07s/it]

auprc: 0.9662099093939771 ± 0.0015274601679476773


In [13]:
file = open('../results/results_bin_best/rfc_model_test_scores.save', 'wb')
pickle.dump(scores, file)
file.close()

file = open('../results/results_bin_best/rfc_model_test_models.save', 'wb')
pickle.dump(final_models, file)
file.close()

### 5. XGBC <a name="5.-xgbc"></a>

In [14]:
import xgboost
from xgboost import XGBClassifier
scores = []
final_models = []
for i in tqdm(range(10)):
    random_state = 42*i    
    X_test, y_test = final_test[i]
    X_train, y_train = shuffle(train_X, train_y, random_state=random_state)
    
    xgbc = XGBClassifier(max_depth=5, learning_rate=0.03, 
                     colsample_bytree=0.9, subsample=0.66,
                     eval_metric='logloss', use_label_encoder=False)
        
    xgbc.fit(X_train, y_train)
    
    y_pred = xgbc.predict_proba(X_test)[:, 1]
    auprc = pr_auc_score(y_test, y_pred)
    
    final_models.append(xgbc)
    scores.append(auprc)   
print(f'auprc: {mean(scores)} ± {stdev(scores)}' )

100%|██████████| 10/10 [02:50<00:00, 17.09s/it]

auprc: 0.9544778611563917 ± 0.0006909616140513083


In [15]:
file = open('../results/results_bin_best/xgbc_model_test_scores.save', 'wb')
pickle.dump(scores, file)
file.close()

file = open('../results/results_bin_best/xgbc_model_test_models.save', 'wb')
pickle.dump(final_models, file)
file.close()

### 6. SVC <a name="6.-svc"></a>

In [17]:
from sklearn.svm import SVC
scores = []
final_models = []
for i in tqdm(range(10)):
    random_state = 42*i    
    X_test, y_test = final_test[i]
    X_train, y_train = shuffle(train_X, train_y, random_state=random_state)
    
    svc = SVC(gamma=0.01, C=5.62, probability=True)
        
    svc.fit(X_train, y_train)
    
    y_pred = svc.predict_proba(X_test)[:, 1]
    auprc = pr_auc_score(y_test, y_pred)
    
    final_models.append(svc)
    scores.append(auprc)   
print(f'auprc: {mean(scores)} ± {stdev(scores)}' )

100%|██████████| 10/10 [8:21:03<00:00, 3006.36s/it] 

auprc: 0.9695386595181812 ± 1.0431814127849422e-06


In [18]:
file = open('../results/results_bin_best/svc_model_test_scores.save', 'wb')
pickle.dump(scores, file)
file.close()

file = open('../results/results_bin_best/svc_model_test_models.save', 'wb')
pickle.dump(final_models, file)
file.close()